In [ ]:
# Numerical and Scientific Computing
import numpy as np
import pandas as pd
import scipy as sp
import math
from numba import jit


from scipy import special
from scipy.integrate import quad
from scipy import integrate
from scipy import stats
from scipy.interpolate import interp1d

# Plotting and Data Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import gridspec
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib as mpl
from matplotlib import ticker, cm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LogNorm
from tabulate import tabulate


from matplotlib import rc

import pyregion

# File Handling
import os
import glob
import csv

# Astronomical

from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
from astroML.stats import binned_statistic_2d


# Other
from tqdm.notebook import tqdm


In [ ]:
threshold = 0.05

z_min, z_max = 0.4, 9.5

physical_width = 35 # h^-1 Mpc

c0 = 3e5
H_0 = 70
Omega_l = 0.7
Omega_m = 0.3

lim_deltaz = 2

def setup(work_path='.'):
    '''
    Set up all of the necessary directories
    '''
    for subdir in ('inputs', 'outputs', 'bin', 
                   'outputs/plots', 'outputs/weights', 'outputs/density'):
        path = os.path.join(work_path, subdir)
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'Built directory: {os.path.abspath(path)}')
    
    outputs_dir = os.path.join(work_path, 'outputs')
    plots_dir = os.path.join(work_path, 'outputs', 'plots')
    inputs_dir = os.path.join(work_path, 'inputs')
    weight_dir = os.path.join(work_path, 'outputs', 'weights')
    density_dir = os.path.join(work_path, 'outputs', 'density')
    return outputs_dir, plots_dir, inputs_dir, weight_dir, density_dir

cat_dir = r"D:\Projects\COSMOS\Results"

outputs_dir, plots_dir, inputs_dir, weights_dir, density_dir = setup(work_path=cat_dir)



# Load the .npy files
weights = np.load(os.path.join(weights_dir, f'weights_unthresholded_normalized_thresh{threshold}_lengh{physical_width}.npy'))
weights_block = np.load(os.path.join(weights_dir, f'weightsBlock_unthresholded_normalized_thresh{threshold}_lengh{physical_width}.npy'))
W = np.load(os.path.join(weights_dir, f'weightsBlock_thresh{threshold}_normalized_lengh{physical_width}.npy'))
normalized_delta_z_median = np.load(os.path.join(weights_dir, f'normalized_delta_z_median_thresh{threshold}_lengh{physical_width}.npy'))
delta_z_median = np.load(os.path.join(weights_dir, f'delta_z_median_thresh{threshold}_lengh{physical_width}.npy'))
count_in_zslice = np.load(os.path.join(weights_dir, f'count_in_zslice_thresh{threshold}_lengh{physical_width}.npy'))
B = "load your bandwidth data here"

#loading Data
Data = "load your data here"

circles = "load your circles data here"
circles_array = circles[['RA (deg)', 'Dec (deg)', 'Radius (deg)']].to_numpy()




def slice_width(z):
    """Calculate the width of each redshift slice of size _physical_width_ (Mpc h^-1)"""
    return physical_width * 100 / c0 * np.sqrt(Omega_m * (1+z)**3 + Omega_l)

def redshift_bins(zmin, zmax):
    """returns the slice centers and widths, given a physical length in (Mpc h^-1) """
    centers = []
    centers.append(zmin + 0.5 * slice_width(zmin))

    i = 0
    while (centers[i] + slice_width(centers[i]) < zmax ):
        centers.append(centers[i] + slice_width(centers[i]))
        i += 1

    centers = np.array(centers)

    "redshift edges"
    edges = np.zeros((len(centers), 2))

    for i in range(0, len(centers)):
        edges[i, 0] = centers[i] - slice_width(centers[i]) / 2
        edges[i, 1] = centers[i] + slice_width(centers[i]) / 2

    return (centers, edges)

slice_centers, z_edges = redshift_bins(z_min, z_max)

z_width = z_edges[:, 1] - z_edges[:, 0]
num_slices = len(slice_centers)


# Use your defined theta
tan = (1.7259 - 1.9644) / (149.66 - 150.31)
theta = -np.arctan(tan)

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import rotate

def plot_slice_summary(sliceNo, slice_center, W, Data, density_dir, circles_array, theta, B):
    
    def rotate_coordinates(ra, dec, theta):
        x = np.cos(theta) * ra + np.sin(theta) * dec
        y = -np.sin(theta) * ra + np.cos(theta) * dec
        return x, y

    def inverse_rotate_coordinates(x, y, theta):
        ra = np.cos(theta) * x - np.sin(theta) * y
        dec = np.sin(theta) * x + np.cos(theta) * y
        return ra, dec

    global_b = B[sliceNo]

    # Load slice file
    pattern = os.path.join(density_dir, f"output_slice_{sliceNo}_*.npz")
    files = [f for f in glob.glob(pattern) if f.endswith('filling_holes.npz')]

    if not files:
        print(f"No file for slice {sliceNo}")
        return
    data = np.load(files[0])

    pts = data['pts']
    adaptive_b = data['adaptive_b']
    boundary_correction = data['boundary_correction']
    density_excess = data['density_excess']

    threshold = 0.01
    actual_mask = (pts[:, 5] > 0) & (pts[:, 2] > threshold)
    artificial_mask = ~actual_mask
    actual_pts = pts[actual_mask]
    artificial_pts = pts[artificial_mask]

    num_actual = len(actual_pts)
    num_artificial = len(artificial_pts)
    total_pts = len(pts)

    # Rotate circles back to RA/Dec
    ra_c, dec_c, rad_c = circles_array[:, 0], circles_array[:, 1], circles_array[:, 2]

    # Inverse rotate pts to RA/Dec
    ra_all, dec_all = inverse_rotate_coordinates(pts[:, 0], pts[:, 1], theta)
    ra_actual, dec_actual = inverse_rotate_coordinates(actual_pts[:, 0], actual_pts[:, 1], theta)
    ra_artificial, dec_artificial = inverse_rotate_coordinates(artificial_pts[:, 0], artificial_pts[:, 1], theta)

    # Rotate density field back
    angle_deg = -np.degrees(theta)
    density_rot = rotate(density_excess, angle=angle_deg, reshape=True, order=1)

    # Estimate extents of rotated RA/Dec from full pts
    x_min, x_max = ra_all.min(), ra_all.max()
    y_min, y_max = dec_all.min(), dec_all.max()

    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'z = {slice_center:.3f} (bin: {sliceNo+1}/{W.shape[1]})', fontsize=16)

    # --- Plot 1: Actual Galaxies ---
    sorted_indices = np.argsort(actual_pts[:, 2])
    sc1 = axs[0, 0].scatter(ra_actual[sorted_indices], dec_actual[sorted_indices],
                            c=actual_pts[sorted_indices, 2], s=1, cmap='Blues',
                            norm=LogNorm(vmin=max(np.min(pts[:, 2]), 1e-3), vmax=1))
    axs[0, 0].set_title(f"Galaxies (N={num_actual})")
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[0, 0].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))
    plt.colorbar(sc1, ax=axs[0, 0], label='weight')

    # --- Plot 2: Artificial Galaxies ---
    sc2 = axs[0, 1].scatter(ra_artificial, dec_artificial, c=artificial_pts[:, 2], s=1, cmap='Blues',
                            norm=LogNorm(vmin=max(np.min(pts[:, 2]), 1e-3), vmax=1))
    axs[0, 1].set_title(f"Artificial Galaxies (N={num_artificial})")
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[0, 1].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))
    plt.colorbar(sc2, ax=axs[0, 1], label='weight')

    # --- Plot 3: All Galaxies ---
    sorted_indices_all = np.argsort(pts[:, 2])
    sc3 = axs[0, 2].scatter(ra_all[sorted_indices_all], dec_all[sorted_indices_all],
                            c=pts[sorted_indices_all, 2], s=1, cmap='Blues',
                            norm=LogNorm(vmin=max(np.min(pts[:, 2]), 1e-3), vmax=1))
    axs[0, 2].set_title(f"All Galaxies (N={total_pts})")
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[0, 2].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))
    plt.colorbar(sc3, ax=axs[0, 2], label='weight')

    # --- Plot 4: Adaptive Bandwidth ---
    ra_b, dec_b = inverse_rotate_coordinates(pts[:, 0], pts[:, 1], theta)
    sc4 = axs[1, 0].scatter(ra_b, dec_b, c=pts[:, 3], s=1, cmap='Spectral')
    axs[1, 0].set_title(f"Adaptive Bandwidth $b_i$ (Global $b$ = {global_b:.4f})")
    plt.colorbar(sc4, ax=axs[1, 0], label=r'$b_i$ [deg]')
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[1, 0].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))

    # --- Plot 5: Boundary Correction ---
    sc5 = axs[1, 1].scatter(ra_b, dec_b, c=pts[:, 4], s=1, cmap='plasma',
                            vmin=np.min(pts[:, 4]), vmax=4)
    axs[1, 1].set_title("Boundary Correction Factor")
    plt.colorbar(sc5, ax=axs[1, 1], label=r'correction factor ($\eta$)')
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[1, 1].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))

    # --- Plot 6: Density Field (Rotated back) ---
    im6 = axs[1, 2].imshow(density_rot,
                           extent=(x_min, x_max, y_min, y_max),
                           origin='lower', aspect='auto', cmap='jet',
                           norm=LogNorm(vmin=np.min(density_excess[density_excess > 0]),
                                        vmax=np.max(density_excess)))
    axs[1, 2].set_title(r"Density Field ($1 + \delta$)")
    plt.colorbar(im6, ax=axs[1, 2], label=r'$1 + \delta$')
    for x0, y0, r in zip(ra_c, dec_c, rad_c):
        axs[1, 2].add_patch(plt.Circle((x0, y0), r, edgecolor='black', facecolor='none',
                                       lw=0.5, linestyle='dashed'))

    # Set axes labels and invert RA
    for ax in axs.flatten():
        ax.set_xlabel('RA [deg]')
        ax.set_ylabel('Dec [deg]')
        ax.invert_xaxis()

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    directory = "path you want to save your plots"
    plt.savefig(os.path.join(directory, f"slice_{sliceNo}_summary_rotated.png"), dpi=300)
    plt.savefig(os.path.join(directory, f"slice_{sliceNo}_summary_rotated.pdf"),dpi=200)
    plt.show()


In [ ]:
# Plot summaries for all slices
for s in range(num_slices):
    plot_slice_summary(sliceNo=s,
                       slice_center=slice_centers[s],
                       W=W,
                       Data=Data,
                       density_dir=density_dir,
                       circles_array=circles_array,
                       theta=theta, B=B)